In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import math
import time

In [2]:
url = 'https://batdongsan.vn/ban-nha/'
data = requests.get(url)

In [3]:
soup = BeautifulSoup(data.text, 'lxml')

In [4]:
num_pages = soup.find("div", {"class":"desc"})
num_pages.text
matches = re.findall(r'\d', num_pages.text)
totals = ''
if len(matches) > 0:
    for number in matches:
        totals += number
totals = int(totals)
totals
total_items = math.ceil(totals/20)
total_items

485

In [5]:
def getHouseInformationToDict(HouseInformations: dict, soup: BeautifulSoup, url: str):
    if(soup.find('div',{'class':'project-global-object-block-003 information-custom'}) and soup.find('div',{'class':'content'})):
        text_info=""
        text_info = soup.find('div',{'class':'project-global-object-block-003 information-custom'}).text.strip() + soup.find('div',{'class':'content'}).text.strip().replace('\n','')
        text_info = text_info.replace("\r"," ")
        text_info = text_info.replace("\n"," ")
        text_info = text_info.replace("\t"," ")
        
        #parse houseTypes & address
        info = soup.find('ul', {'class':'uk-breadcrumb'}).text.strip().replace('\n',',').split(',')
        HouseInformations['address'].append(info[3]+", " +info[2])
        HouseInformations['houseTypes'].append(info[1])
        
        
        # parse area
        z = re.findall(r"(?<=Diện tích: )(\d+(\.\d+)?)", text_info) or re.findall(r"DT\s+(\d+(\.\d+)?)", text_info, re.IGNORECASE)
        
        if z:
            HouseInformations['area'].append(z[0][0])
        else:
            HouseInformations['area'].append(np.nan)

        #parse floor & num_bedrooms

        pattern = r"(\d+)\s?(?:Tầng|Tang|tang| tang|tang |tầng)"
        matches = ""
        matches = re.findall(pattern, text_info, re.IGNORECASE)
        # print(text_info)
        # print(matches)
        if matches:
            HouseInformations['num_floors'].append(matches[0])
        else:
            # print("else check")
            pattern = r"Tầng\s+(\d+(?:\s*,\s*\d+)*)\b|\bTầng\s+(\d+)\b"
            matches = re.findall(pattern, text_info, re.IGNORECASE)
            # print("else check: ", matches)
            if matches:
                numbers = []
                for match in matches:
                    if match[0] != '':
                        numbers.extend([int(num) for num in re.findall(r'\d+', match[0])])
                    elif match[1] != '':
                        numbers.append(int(match[1]))
                max_number = max(numbers)
                HouseInformations['num_floors'].append(str(max_number))
            else:
                    # print("Lầu check")
                    # pattern = r'\b(\d+)\b\s*(?:T|lầu)'
                    pattern = r'(?:\D+|\s+)(\d+)\s*(?:T|lầu)\b'
                    matches = re.findall(pattern, text_info)
                    # print("maches", matches)
                    if matches:
                        numbers = [int(match) for match in matches]
                        # print(numbers)
                        # print(matches)
                        HouseInformations['num_floors'].append(str(max(numbers)))
                    else:
                        HouseInformations['num_floors'].append(np.nan)
                

        
        
        z = (re.search(r"Phòng ngủ: (\d+) PN", text_info, re.IGNORECASE) or re.search(r"(\d+) PN", text_info)
            or re.search(r"(\d+)PN", text_info)
            or re.search(r"(\d+) phòng ngủ", text_info, re.IGNORECASE)
            or re.search(r"(\d+)pn", text_info)
            or re.search(r"(\d+) pn", text_info))
        if z != None:
            HouseInformations['bedrooms'].append(z.group(1))
        else:
            HouseInformations['bedrooms'].append(np.nan)
        
        
        # parse price
        price = soup.find('div',{'class':'project-global-object-block-003 information-custom'}).find('strong',{'class':'price'}).text.strip()
        if price == 'Thỏa thuận':
            z = (re.search(r"(\d+(?:\.\d+)?(?:,\d+)?) Tỷ", text_info) or re.search(r"(\d+(?:\.\d+)?(?:,\d+)?) tỷ", text_info))

            if z != None:
                HouseInformations['price'].append(z.group(1))     # TH: Tỷ
            else:
                z = (re.search(r"(\d+(?:\.\d+)?(?:,\d+)?) triệu", text_info) or re.search(r"(\d+(?:\.\d+)?(?:,\d+)?) tr", text_info)) # cho nếu nhà giá triệu
                if z != None:
                    HouseInformations['price'].append(str(float(z.group(1))/1000))
                else:
                    HouseInformations['price'].append(z)
        else:
            z = re.search(r"(\d+(?:\.\d+)?(?:,\d+)?) tỷ", price)
            if z != None:
                if float(z.group(1)) > 1000 and float(z.group(1)) < 10000:
                    HouseInformations['price'].append(str(float(z.group(1))/1000))
                elif float(z.group(1)) > 100000:
                    HouseInformations['price'].append(str(float(z.group(1))/1000000))
                else:
                    HouseInformations['price'].append(z.group(1))
            else:
                z = re.search(r"(\d+(?:\.\d+)?(?:,\d+)?) triệu", price)
                if z != None:
                    if (float(z.group(1)) < 100):
                        HouseInformations['price'].append(z.group(1))
                    else:
                        HouseInformations['price'].append(str(float(z.group(1))/1000))
                else:
                    HouseInformations['price'].append(z)
        HouseInformations['url'].append(url)
    else:
        pass

In [6]:
soup_temp = soup
url_pages = 'https://batdongsan.vn/ban-nha/'
HouseInformations = {"houseTypes":[],"area":[], "num_floors":[],"bedrooms":[], "address":[], "price":[], 'url':[]}
for i in range(2, total_items+1): #start from 'https://batdongsan.vn/ban-nha/' after that to /p2 total_items+1 as 3 run only first page
    print(i)
    data_list = soup_temp.find('div', {'class':'datalist'})
    for item in data_list.find_all('div', {'class':'name'}):
        house_html = requests.get(item.a['href']) # link từng cái nhà 
        if house_html:
            house_soup = BeautifulSoup(house_html.text, 'lxml')
            getHouseInformationToDict(HouseInformations, house_soup, item.a['href'])
    print(url_pages +'p%d' %i)
    data_temp = requests.get(url_pages +'p%d' %i) # url_pages +'p%d' %i next page link
    soup_temp = BeautifulSoup(data_temp.text, 'lxml')

2
https://batdongsan.vn/ban-nha/p2


In [7]:
for i in HouseInformations:
    print(len(HouseInformations[i]))

20
20
20
20
20
20
20


In [8]:
df = pd.DataFrame.from_dict(HouseInformations)

In [9]:
df

,houseTypes,area,num_floors,bedrooms,address,price,url
0,Bán Nhà riêng,130,3,7,"Quận 12, TP Hồ Chí Minh",9.2,https://batdongsan.vn/ban-biet-thu-san-vuon-qu...
1,Bán Nhà riêng,65,NaN,2,"Ngũ Hành Sơn, Đà Nẵng",2.9,https://batdongsan.vn/29-ty-dien-tich-6580m2-n...
2,Bán Nhà riêng,90,5,5,"Hai Bà Trưng, Hà Nội",14,https://batdongsan.vn/ban-nha-kim-nguu-phan-lo...
3,Bán Nhà riêng,NaN,5,5,"Quận 3, TP Hồ Chí Minh",0.001,https://batdongsan.vn/quan-3-le-van-sy-5-tang-...
4,Bán Nhà riêng,30,1,2,"Gò Vấp, TP Hồ Chí Minh",3.7,https://batdongsan.vn/nha-moi-full-noi-that-hx...
5,Bán Nhà riêng,38,3,3,"Bình Thạnh, TP Hồ Chí Minh",4.4,https://batdongsan.vn/ban-gap-nha-dep-dien-bie...
6,Bán Nhà riêng,45,2,2,"Tân Phú, TP Hồ Chí Minh",3.9,https://batdongsan.vn/nha-moi-cach-1-can-ra-ma...
7,Bán Nhà riêng,45,2,2,"Tân Phú, TP Hồ Chí Minh",3.9,https://batdongsan.vn/nha-moi-cach-1-can-ra-ma...
8,Bán Nhà riêng,51,6,4,"Gò Vấp, TP Hồ Chí Minh",12.7,https://batdongsan.vn/go-vap-khu-nha-cao-tang-...
9,Bán Nhà mặt phố,50,4,4,"Đống Đa, Hà Nội",25,https://batdongsan.vn/ban-nha-lo-goc-mat-pho-l...
